In [363]:
import pandas as pd
import datetime

In [364]:
# set display 
pd.set_option('display.max.columns', 100)
pd.set_option('display.max.rows', 1000)

In [365]:
# 파일 불러오기
file = pd.read_csv('C:/Users/user/Desktop/직원 현황 테스트/210527_사원명부.csv', encoding='utf-8')

In [366]:
# 필요한 컬럼명 정의
arrColumns=['사원','사번', '주민등록번호M', '성별', '나이' ,'부서','직급', '직위', '직책', '사원구분', '입사일','현부서근무일', '현직급근무일', '현직책근무일',
            '2015년 종합근무성적평가 결과', '2016년 종합근무성적평가 결과', '2017년 종합근무성적평가 결과','2018년 종합근무성적평가 결과','2019년 종합근무성적평가 결과','2020년 종합근무성적평가 결과']

In [367]:
# 필요한 컬럼만 필터
df = file[arrColumns]
# 아무런 정보도 없는 행 삭제
df = df.loc[df['사원'].dropna().index]

In [368]:
# 발령내역 csv 파일 불러온 후 필요한 컬럼만 필터
move_df = pd.read_csv('C:/Users/user/Desktop/직원 현황 테스트/210715_발령내역.csv', encoding='utf-8')
col=['사원','사번','발령일','소속부서','직급','발령명', '비고']
move_df = move_df[col]

In [369]:
# 발령이력 정리(인자는 df)
def create_history(temp) :
    # df를 역순으로 재배치
    temp = temp[::-1]
    # df의 인덱스 추출
    temp_index = temp.index

    elements = []
    for k in temp_index :
        # 각 발령 정보 추출
        date = temp.loc[k, '발령일']
        depart = temp.loc[k, '소속부서']
        grade = temp.loc[k, '직급']
        order_name = temp.loc[k, '발령명']
        etc = temp.loc[k, '비고']
        # 비고란에 내용 없으면 na값 대신 공백 처리
        if pd.isna(etc):
            etc = ""
        # 한줄 메세지 생성하고 리스트에 저장
        element = '{0}  {1} ({2})   {3}'.format(date, depart, grade, etc)
        elements.append(element)
    # 리스트에 모여있는 각 한줄 메세지들을 하나의 단락으로 병합
    final = "\n".join(elements)
    return final

In [370]:
historys = []
# 사원명부에 있는 사원 별로 발령이력을 하나씩 생성
for k in range(0, len(df)) :
    name = df.loc[k, '사원']
    temp = move_df[move_df['사원']==name]
    history = create_history(temp)
    historys.append(history)
    
# 마지막 결과물을 df에 발령일력 컬럼 생성
df['주요 발령사항'] = historys

In [371]:
# 근무성적과 관련된 컬럼 정의
col_ = ['2015년 종합근무성적평가 결과', '2016년 종합근무성적평가 결과', '2017년 종합근무성적평가 결과','2018년 종합근무성적평가 결과','2019년 종합근무성적평가 결과','2020년 종합근무성적평가 결과']

df.columns
for c in col_ :
    # 각 연도별 근무성적 컬럼이 비어 있거나 '0'으로 표기 되어 있으면 '-'표시로 변경
    df[c].fillna("-", inplace=True)
    df[c]=df[c].apply(lambda x: "-" if x == '0' else x)

# 각 연도별 근무성적 컬럼을 하나의 컬럼으로 병합(각 결과 사이는 줄바꿈)
df['근무성적'] = '2015년 '+df['2015년 종합근무성적평가 결과']+"\n"+'2016년 '+df['2016년 종합근무성적평가 결과']+"\n"+'2017년 '+df['2017년 종합근무성적평가 결과']+"\n"+'2018년 '+df['2018년 종합근무성적평가 결과']+"\n"+'2019년 '+df['2019년 종합근무성적평가 결과']+"\n"+'2020년 '+df['2020년 종합근무성적평가 결과']

df = df.drop(col_, axis=1)

In [372]:
# 외부경력 파일 불러오고 필요한 컬럼만 필터
# (주의사항) 외부경력에 특수기호 등이 있을 경우 나중에 csv로 저장하는 과정에서 인코딩 에러가 발생하기 때문에 전처리 필요
career_df = pd.read_csv('C:/Users/user/Desktop/직원 현황 테스트/210715_외부경력.csv', encoding='utf-8')
col=['사원','사번','직장명','입사일','퇴사일','기타담당업무']
career_df = career_df[col]

In [373]:
# 외부경력 정리(인자는 df)
def create_career(temp) :
    # df를 역순으로 재배치
    temp = temp[::-1]
    # df의 인덱스 추출
    temp_index = temp.index

    elements = []
    for k in temp_index :
        # 각 외부경력 정보 추출
        s_date = temp.loc[k, '입사일']
        e_date = temp.loc[k, '퇴사일']
        firm = temp.loc[k, '직장명']
        detail = temp.loc[k, '기타담당업무']
        # 비고란에 내용 없으면 na값 대신 공백 처리
        if pd.isna(detail):
            detail = ""
        # 한줄 메세지 생성하고 리스트에 저장
        element = '{0} ~ {1}  ({2})  {3}'.format(s_date, e_date, firm, detail)
        elements.append(element)
    # 리스트에 모여있는 각 한줄 메세지들을 하나의 단락으로 병합
    final = "\n".join(elements)
    return final

In [374]:
careers = [] 
# 사원명부에 있는 사원 별로 외부경력 이력을 하나씩 생성
for k in range(0, len(df)) :
    name = df.loc[k, '사원']
    temp = career_df[career_df['사원']==name]
    career = create_career(temp)
    careers.append(career)
# df에 외부경력 이력 삽입
df['주요 외부경력'] = careers

In [375]:
# 학사정보 csv 파일 불러온 후 필요한 컬럼만 필터 
# (주의) 학사정보 파일은 encoding='CP949'로 불러와야함
accademic_df = pd.read_csv('C:/Users/user/Desktop/직원 현황 테스트/210716_학사정보.csv', encoding='CP949')
col=['사원','사번','학교','전공학과','입학년월','졸업년월', '학위구분']
accademic_df = accademic_df[col]

In [376]:
accademic_df['order'] = accademic_df['학위구분']
accademic_df['order'].unique()

array(['학사', '석사', '전문학사', '박사', '편입', '박사수료', '석사수료', '고졸', '대퇴', '중퇴'],
      dtype=object)

In [377]:
# 학사 순서 설정
accademic_df['order']=accademic_df['order'].apply(lambda x : 1 if x=='고졸' else 2 if x=='중퇴' else 3 if  x=='대퇴' else 4 if  x=='전문학사' else 5 if  x=='편입' else 6 if  x=='학사' else 7 if  x=='석사수료' else 8 if  x=='석사' else 9 if  x=='박사수료' else 10)
# 학사 순서대로 재배열
accademic_df=accademic_df.sort_values(by='order')
# create_accademic 안에서 인덱스 순서대로 for loop를 돌기 때문에 index를 reset함
accademic_df=accademic_df.reset_index(drop=True)

In [378]:
# 외부경력 정리(인자는 df)
def create_accademic(temp) :
    # df를 역순으로 재배치
#     temp = temp[::-1]
    # df의 인덱스 추출
    temp_index = temp.index

    elements = []
    for k in temp_index :
        # 각 학사정보 추출
        kind = temp.loc[k, '학위구분']
        accademy = temp.loc[k, '학교']
        major = temp.loc[k, '전공학과']
        # 전공학과 내용 없으면 na값 대신 공백 처리
        if pd.isna(major):
            major = ""
        # 한줄 메세지 생성하고 리스트에 저장
        element = '({0})  {1}  {2}'.format(kind, accademy, major)
        elements.append(element)
    # 리스트에 모여있는 각 한줄 메세지들을 하나의 단락으로 병합
    final = "\n".join(elements)
    return final

In [379]:
accademics = [] 
# 사원명부에 있는 사원 별로 외부경력 이력을 하나씩 생성
for k in range(0, len(df)) :
    name = df.loc[k, '사원']
    temp = accademic_df[accademic_df['사원']==name]
    accademic = create_accademic(temp)
    accademics.append(accademic)
# df에 외부경력 이력 삽입
df['학사 정보'] = accademics

In [380]:
# df 컬럼 순서 재배열
df=df[['사원', '사번', '주민등록번호M', '성별', '나이', '부서', '직급', '직위', '직책', '사원구분',
       '입사일', '현부서근무일', '현직급근무일', '현직책근무일', '주요 발령사항', '근무성적', '학사 정보', '주요 외부경력']]
# df저장
df.to_csv('C:/Users/user/Desktop/직원 현황 테스트/210715_인사종합정보.csv', encoding='cp949')

In [381]:
def load_file_utf8(path, file_name) :
    file = pd.read_csv('{0}/{1}.csv'.format(path, file_name), encoding='utf-8')
    return file

In [382]:
def delet(df, path):
    rubish = load_file_utf8(path, '지우기 목록')
    indice=[]
    for rubish in rubish['사원'] :
        index=df[df['사원']==rubish].index[0]
        indice.append(index)
    df=df.drop(indice)
    return df

In [383]:
def tem_employee(df, path):
    tem = load_file_utf8(path, '계약직 목록')
    for t in tem['사원'] :
        # df[df['사원']==t]['직급'] = '계약직'
        df.loc[(df['사원'] == t), '직급'] = '계약직'
        df
    return df

In [384]:
def arrange_department(df, date, path) :
    order_df = load_file_utf8(path, '직제순서')
    order_df['시점']=pd.to_datetime(order_df['시점'], format='%Y-%m-%d')
    order_df=order_df[order_df['시점']==date]
    df_merge = pd.merge(order_df, df, left_on='부서', right_on='부서')
    df_merge = df_merge.sort_values(by=['직제순서'])
    df_merge=df_merge.drop(['시점', '직제순서'], axis=1)
    return df_merge

In [385]:
path = 'C:/Users/user/Desktop/직원 현황 테스트'
df = delet(df, path)
df = tem_employee(df, path)
date = datetime.datetime(2021, 1, 1)
df = arrange_department(df, date, path)

In [386]:
# df 컬럼 순서 재배열
df=df[['사원', '사번', '주민등록번호M', '성별', '나이', '부서', '직급', '직위', '직책', '사원구분',
       '입사일', '현부서근무일', '현직급근무일', '현직책근무일', '주요 발령사항', '근무성적', '학사 정보', '주요 외부경력']]
# df저장
df.to_csv('C:/Users/user/Desktop/직원 현황 테스트/210715_인사종합정보.csv', encoding='cp949')